# Comparative Acquisition of Complex Sentence Structures in Bilingual and Monolingual Children

## 1. Load all datasets

The project contains multiple CHILDES datasets organized as follows:

1. **English-MiamiBiling/**: Bilingual English data (2 & 5-year-olds, audio/no-audio)
2. **English-MiamiMono/**: Monolingual English data (2 & 5-year-olds)  
3. **Spanish-MiamiBiling/**: Bilingual Spanish data (2 & 5-year-olds, audio/no-audio)
4. **Hoff/**: Additional dataset with English/Spanish bilingual and monolingual data (age groups 2.5, 3, 3.5)

In [1]:
import pylangacq as pla
import os
import glob
import pandas as pd

In [2]:
def robust_load_cha(file_path):
    """
    Robust function to load .cha files with fallback strategies
    Returns: (success, chat_object_or_utterance_count, method_used)
    """
    
    # Strategy 1: Try normal pylangacq loading
    try:
        chat = pla.read_chat(file_path)
        return True, chat, 'pylangacq'
    except Exception as e1:
        # Strategy 2: Try with explicit encoding
        try:
            chat = pla.read_chat(file_path, encoding='utf-8')
            return True, chat, 'pylangacq_utf8'
        except Exception as e2:
            # Strategy 3: Raw parsing for basic utterance count
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                lines = content.split('\n')
                chi_utterances = [line for line in lines if line.strip().startswith('*CHI:')]
                return False, chi_utterances, 'raw_parsing'
            except Exception as e3:
                return False, 0, 'failed'

def load_directory_structure():
    """
    Comprehensively load all .cha files from all directories
    """
    
    # Define all dataset directories
    datasets = {
        'English-MiamiBiling': {
            'bleng2-audio': 'English_Bilingual_Age2_Audio',
            'bleng2-noaudio': 'English_Bilingual_Age2_NoAudio', 
            'bleng5-audio': 'English_Bilingual_Age5_Audio',
            'bleng5-noaudio': 'English_Bilingual_Age5_NoAudio'
        },
        'English-MiamiMono': {
            'mleng2': 'English_Monolingual_Age2',
            'mleng5': 'English_Monolingual_Age5'
        },
        'Spanish-MiamiBiling': {
            'blspan2-audio': 'Spanish_Bilingual_Age2_Audio',
            'blspan2-noaudio': 'Spanish_Bilingual_Age2_NoAudio',
            'blspan5-audio': 'Spanish_Bilingual_Age5_Audio', 
            'blspan5-noaudio': 'Spanish_Bilingual_Age5_NoAudio'
        },
        'Hoff': {
            'biling-eng/2.5': 'Hoff_Bilingual_English_Age2.5',
            'biling-eng/3': 'Hoff_Bilingual_English_Age3',
            'biling-eng/3.5': 'Hoff_Bilingual_English_Age3.5',
            'biling-spa/2.5': 'Hoff_Bilingual_Spanish_Age2.5',
            'biling-spa/3': 'Hoff_Bilingual_Spanish_Age3',
            'biling-spa/3.5': 'Hoff_Bilingual_Spanish_Age3.5',
            'mono/2.5': 'Hoff_Monolingual_Age2.5',
            'mono/3': 'Hoff_Monolingual_Age3',
            'mono/3.5': 'Hoff_Monolingual_Age3.5'
        },

        'Spanish-Ornat':{
            '03': 'Spanish_Ornat_Monolingual_Age3',
            '04': 'Spanish_Ornat_Monolingual_Age4',
        },
        
        'Spanish-Sebastian':{
            '03': 'Spanish_Sebastian_Monolingual_Age3',
            '04': 'Spanish_Sebastian_Monolingual_Age4'
   
        }
    }
    
    all_datasets = {}
    summary_stats = []
    
    print("Loading all datasets...")
    print("=" * 60)
    
    for main_dir, subdirs in datasets.items():
        for subdir, dataset_name in subdirs.items():
            dir_path = os.path.join(main_dir, subdir)
            
            if not os.path.exists(dir_path):
                print(f"⚠️  Directory not found: {dir_path}")
                continue
                
            # Find all .cha files
            cha_files = glob.glob(os.path.join(dir_path, "*.cha"))
            
            if not cha_files:
                print(f"📁 {dataset_name}: No .cha files found")
                continue
            
            # Load files
            dataset_files = []
            successful_loads = 0
            partial_loads = 0
            failed_loads = 0
            
            for file_path in cha_files:
                success, result, method = robust_load_cha(file_path)
                
                file_info = {
                    'filename': os.path.basename(file_path),
                    'filepath': file_path,
                    'dataset': dataset_name,
                    'main_category': main_dir,
                    'subcategory': subdir,
                    'success': success,
                    'method': method
                }
                
                if success:
                    file_info['chat_object'] = result
                    file_info['participants'] = result.participants()
                    file_info['utterance_count'] = len(result.utterances())
                    successful_loads += 1
                elif method == 'raw_parsing':
                    file_info['utterance_count'] = len(result)
                    file_info['chi_utterances'] = result
                    partial_loads += 1
                else:
                    failed_loads += 1
                
                dataset_files.append(file_info)
            
            # Store dataset
            all_datasets[dataset_name] = dataset_files
            
            # Print summary
            total_files = len(cha_files)
            print(f"📊 {dataset_name}")
            print(f"   Total files: {total_files}")
            print(f"   ✅ Fully loaded: {successful_loads}")
            print(f"   🔶 Partial (raw): {partial_loads}")
            print(f"   ❌ Failed: {failed_loads}")
            print()
            
            # Add to summary stats
            summary_stats.append({
                'dataset': dataset_name,
                'main_category': main_dir,
                'subcategory': subdir,
                'total_files': total_files,
                'successful': successful_loads,
                'partial': partial_loads,
                'failed': failed_loads,
                'success_rate': (successful_loads / total_files * 100) if total_files > 0 else 0
            })
    
    return all_datasets, pd.DataFrame(summary_stats)

In [ ]:
all_data, summary_df = load_directory_structure()

Loading all datasets...
📊 English_Bilingual_Age2_Audio
   Total files: 21
   ✅ Fully loaded: 12
   🔶 Partial (raw): 9
   ❌ Failed: 0

📊 English_Bilingual_Age2_NoAudio
   Total files: 66
   ✅ Fully loaded: 33
   🔶 Partial (raw): 33
   ❌ Failed: 0

📊 English_Bilingual_Age5_Audio
   Total files: 16
   ✅ Fully loaded: 7
   🔶 Partial (raw): 9
   ❌ Failed: 0

📊 English_Bilingual_Age5_NoAudio
   Total files: 73
   ✅ Fully loaded: 47
   🔶 Partial (raw): 26
   ❌ Failed: 0

📊 English_Monolingual_Age2
   Total files: 43
   ✅ Fully loaded: 19
   🔶 Partial (raw): 24
   ❌ Failed: 0

📊 English_Monolingual_Age5
   Total files: 46
   ✅ Fully loaded: 29
   🔶 Partial (raw): 17
   ❌ Failed: 0

📊 Spanish_Bilingual_Age2_Audio
   Total files: 19
   ✅ Fully loaded: 13
   🔶 Partial (raw): 6
   ❌ Failed: 0

📊 Spanish_Bilingual_Age2_NoAudio
   Total files: 68
   ✅ Fully loaded: 30
   🔶 Partial (raw): 38
   ❌ Failed: 0

📊 Spanish_Bilingual_Age5_Audio
   Total files: 16
   ✅ Fully loaded: 11
   🔶 Partial (raw): 5


In [35]:
# Display comprehensive summary
print("\n" + "="*80)
print("📈 COMPREHENSIVE DATASET SUMMARY")
print("="*80)

# Overall statistics
total_datasets = len(all_data)
total_files = sum(len(files) for files in all_data.values())
total_successful = summary_df['successful'].sum()
total_partial = summary_df['partial'].sum() 
total_failed = summary_df['failed'].sum()

print(f"📁 Total datasets: {total_datasets}")
print(f"📄 Total files: {total_files}")
print(f"✅ Fully loaded: {total_successful} ({total_successful/total_files*100:.1f}%)")
print(f"🔶 Partially loaded: {total_partial} ({total_partial/total_files*100:.1f}%)")
print(f"❌ Failed: {total_failed} ({total_failed/total_files*100:.1f}%)")
print(f"📊 Overall success rate: {(total_successful+total_partial)/total_files*100:.1f}%")

print("\n" + "="*80)
print("📋 DETAILED BREAKDOWN BY DATASET")
print("="*80)

# Sort by main category for better organization
summary_df_sorted = summary_df.sort_values(['main_category', 'subcategory'])
print(summary_df_sorted.to_string(index=False))

print("\n" + "="*80)
print("🔍 DATASET ACCESS EXAMPLES")
print("="*80)

# Show examples of how to access data
for dataset_name, files in list(all_data.items())[:3]:  # Show first 3 datasets
    print(f"\n📚 {dataset_name}:")
    print(f"   Access: all_data['{dataset_name}']")
    print(f"   Files: {len(files)}")
    
    if files:
        # Show example file
        example_file = files[0]
        print(f"   Example file: {example_file['filename']}")
        if example_file['success']:
            print(f"   Participants: {example_file['participants']}")
            print(f"   Utterances: {example_file['utterance_count']}")
        else:
            print(f"   Utterances (raw): {example_file.get('utterance_count', 'N/A')}")


📈 COMPREHENSIVE DATASET SUMMARY
📁 Total datasets: 23
📄 Total files: 853
✅ Fully loaded: 646 (75.7%)
🔶 Partially loaded: 207 (24.3%)
❌ Failed: 0 (0.0%)
📊 Overall success rate: 100.0%

📋 DETAILED BREAKDOWN BY DATASET
                           dataset       main_category     subcategory  total_files  successful  partial  failed  success_rate
      English_Bilingual_Age2_Audio English-MiamiBiling    bleng2-audio           21          12        9       0     57.142857
    English_Bilingual_Age2_NoAudio English-MiamiBiling  bleng2-noaudio           66          33       33       0     50.000000
      English_Bilingual_Age5_Audio English-MiamiBiling    bleng5-audio           16           7        9       0     43.750000
    English_Bilingual_Age5_NoAudio English-MiamiBiling  bleng5-noaudio           73          47       26       0     64.383562
          English_Monolingual_Age2   English-MiamiMono          mleng2           43          19       24       0     44.186047
          English_Mono

# Parse Datasets

In [ ]:
import pandas as pd
import textstat
from tqdm import tqdm

# To switch languages:
language = 'en'  # example: 'en' for English, 'es' for Spanish

textstat.set_lang(language)

In [ ]:
def get_plain_utterances_from_dataset(dataset, calc_scores=True):
    utterances_dictionary = {}   
    for file in tqdm(all_data[dataset]):
        plain_utterances = []
        if file['success'] == True:
            for ut in file['chat_object'].utterances():
                plain_utterances.append(' '.join([token.word for token in ut.tokens]))

                if calc_scores:
                    scores = [
                        file['dataset'],
                        file['filename'],
                        [textstat.flesch_reading_ease(x) for x in plain_utterances],
                        [textstat.fernandez_huerta(x) for x in plain_utterances],
                    ]
                    
                    utterances_dictionary[file['filename']] = [plain_utterances, scores]
                else:
                    utterances_dictionary[file['filename']] = [plain_utterances]

    return utterances_dictionary

def utterances_all_files(datasets, calc_scores=True):
    total_utterances_dictionary = {}
    for dataset in datasets:
        utterances = get_plain_utterances_from_dataset(dataset, calc_scores=True)
        total_utterances_dictionary[dataset] = utterances

    return total_utterances_dictionary

In [ ]:
all_utterances = utterances_all_files([x for x in summary_df['dataset']])

100%|██████████| 12/12 [00:00<00:00, 1332.86it/s]


In [39]:
all_utterances.keys()

dict_keys(['English_Bilingual_Age2_Audio', 'English_Bilingual_Age2_NoAudio', 'English_Bilingual_Age5_Audio', 'English_Bilingual_Age5_NoAudio', 'English_Monolingual_Age2', 'English_Monolingual_Age5', 'Spanish_Bilingual_Age2_Audio', 'Spanish_Bilingual_Age2_NoAudio', 'Spanish_Bilingual_Age5_Audio', 'Spanish_Bilingual_Age5_NoAudio', 'Hoff_Bilingual_English_Age2.5', 'Hoff_Bilingual_English_Age3', 'Hoff_Bilingual_English_Age3.5', 'Hoff_Bilingual_Spanish_Age2.5', 'Hoff_Bilingual_Spanish_Age3', 'Hoff_Bilingual_Spanish_Age3.5', 'Hoff_Monolingual_Age2.5', 'Hoff_Monolingual_Age3', 'Hoff_Monolingual_Age3.5', 'Spanish_Ornat_Monolingual_Age3', 'Spanish_Ornat_Monolingual_Age4', 'Spanish_Sebastian_Monolingual_Age3', 'Spanish_Sebastian_Monolingual_Age4'])

In [40]:
import csv
import json

csv_filename = "all_utterances.csv"
fieldnames = ['dataset', 'filename', 'plain_utterances', 'scores']

with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for dataset, files in all_utterances.items():
        # files is a dict: filename -> [plain_utterances, scores] (scores may be absent)
        for filename, value in files.items():
            plain = value[0] if len(value) > 0 else []
            scores = value[1] if len(value) > 1 else None
            writer.writerow({
                'dataset': dataset,
                'filename': filename,
                'plain_utterances': json.dumps(plain, ensure_ascii=False),
                'scores': json.dumps(scores, ensure_ascii=False)
            })